In [ ]:
# Standard Section 1: Introduction to Web Scraping

In [ ]:
## some part of the codes is not working yet

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("http://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")

import requests
from bs4 import BeautifulSoup
from IPython.display import HTML

In [4]:
# Setting up 'requests' to make HTTPS requests properly takes some extra steps... we'll skip them for now.
requests.packages.urllib3.disable_warnings()

In [ ]:
# Ingesting data 

In [6]:
url = "https://www.nobelprize.org/prizes/lists/all-nobel-prizes/"
# All 3 of these do the same thing. The third is my (KCA's) favorite new feature of Python 3.6.
wayback_query_url = 'http://archive.org/wayback/available?url={}'.format(url)
wayback_query_url = 'http://archive.org/wayback/available?url={url}'.format(url=url)
# wayback_query_url = f'http://archive.org/wayback/available?url={url}'
r = requests.get(wayback_query_url)

In [7]:
r.text

'{"archived_snapshots": {"closest": {"available": true, "status": "200", "timestamp": "20190826213936", "url": "http://web.archive.org/web/20190826213936/https://www.nobelprize.org/prizes/lists/all-nobel-prizes"}}, "url": "https://www.nobelprize.org/prizes/lists/all-nobel-prizes/"}'

In [8]:
import json
json.loads(r.text)

{'archived_snapshots': {'closest': {'available': True,
   'status': '200',
   'timestamp': '20190826213936',
   'url': 'http://web.archive.org/web/20190826213936/https://www.nobelprize.org/prizes/lists/all-nobel-prizes'}},
 'url': 'https://www.nobelprize.org/prizes/lists/all-nobel-prizes/'}

In [9]:
response_json = r.json()
response_json

{'archived_snapshots': {'closest': {'available': True,
   'status': '200',
   'timestamp': '20190826213936',
   'url': 'http://web.archive.org/web/20190826213936/https://www.nobelprize.org/prizes/lists/all-nobel-prizes'}},
 'url': 'https://www.nobelprize.org/prizes/lists/all-nobel-prizes/'}

In [10]:
# get the snapshot URL?
snapshot_url = response_json['archived_snapshots']['closest']['url']
snapshot = requests.get(snapshot_url)

In [11]:
raw_html = snapshot.text
print(raw_html[:500])

	<!DOCTYPE html>

	<html lang="en-US" class="no-js">

	<head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app57.us.archive.org';v.server_ms=660;archive_analytics.send_pageview({});});</script><script type="text/javascript" src="/_static/js/ait-client-rewrite.js" charset="utf-8"></script>
<script type


In [ ]:
# Parse the HTML with BeautifulSoup

In [12]:
soup = BeautifulSoup(raw_html, 'html.parser')

In [13]:
# Print a cleaned-up version of raw HTML
print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="no-js" lang="en-US">
 <head>
  <script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript">
  </script>
  <script type="text/javascript">
   window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app57.us.archive.org';v.server_ms=660;archive_analytics.send_pageview({});});
  </script>
  <script charset="utf-8" src="/_static/js/ait-client-rewrite.js" type="text/javascript">
  </scr


In [14]:
# Find the first “title” object
soup.select_one('title')

<title>All Nobel Prizes</title>

In [15]:
soup.select_one('title').text

'All Nobel Prizes'

In [16]:
# Extracting award data

In [17]:
award_nodes = soup.select('.by_year')
len(award_nodes)

641

In [18]:
# take a look
award_nodes[200:202]

[<div class="by_year">
 <h3><a href="http://web.archive.org/web/20190826213936/https://www.nobelprize.org/nobel_prizes/chemistry/laureates/1985/">The Nobel Prize in Chemistry 1985</a></h3>
 <p><a href="http://web.archive.org/web/20190826213936/https://www.nobelprize.org/nobel_prizes/chemistry/laureates/1985/hauptman-facts.html">Herbert A. Hauptman</a> and <a href="http://web.archive.org/web/20190826213936/https://www.nobelprize.org/nobel_prizes/chemistry/laureates/1985/karle-facts.html">Jerome Karle</a><br/>
 “for their outstanding achievements in the development of direct methods for the determination of crystal structures”</p>
 </div>, <div class="by_year">
 <h3><a href="http://web.archive.org/web/20190826213936/https://www.nobelprize.org/nobel_prizes/medicine/laureates/1985/">The Nobel Prize in Physiology or Medicine 1985</a></h3>
 <p><a href="http://web.archive.org/web/20190826213936/https://www.nobelprize.org/nobel_prizes/medicine/laureates/1985/brown-facts.html">Michael S. Brown<

In [19]:
award_node = award_nodes[200]

In [20]:
HTML(award_node.prettify())

In [21]:
[node.text for node in award_node.select('h6 a')]

[]

In [22]:
award_node.select_one('p').text

'Herbert A. Hauptman and Jerome Karle\n“for their outstanding achievements in the development of direct methods for the determination of crystal structures”'

In [23]:
print(json.dumps(award_node.select_one('p').text))

"Herbert A. Hauptman and Jerome Karle\n\u201cfor their outstanding achievements in the development of direct methods for the determination of crystal structures\u201d"


In [24]:
award_node.select_one('p').text.lstrip('\u201c').rstrip('\u201d')

'Herbert A. Hauptman and Jerome Karle\n“for their outstanding achievements in the development of direct methods for the determination of crystal structures'

In [25]:
def get_award_motivation(award_node):
    award_node = award_node.select_one('p')
    if not award_node:
        return None
    return award_node.text.lstrip('\u201c').rstrip('\u201d')

In [26]:
def get_recipients(award_node):
    return [node.text for node in award_node.select('h6 a')]

In [29]:
def get_award_title(award_node):
    return soup.select_one('title').text

In [50]:
def get_award_year(award_node):
    return soup.select_one('.by_year')

In [51]:
# Now let's get all of the awards.
awards = []
for award_node in soup.select('.by_year'):
    recipients = get_recipients(award_node)
    awards.append(dict(
        title=get_award_title(award_node),
        year=get_award_year(award_node),
        recipients=recipients,
        num_recipients=len(recipients),
        motivation=get_award_motivation(award_node)
    ))

In [52]:
df_awards_raw = pd.DataFrame(awards)

In [53]:
df_awards_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 5 columns):
motivation        640 non-null object
num_recipients    641 non-null int64
recipients        641 non-null object
title             641 non-null object
year              641 non-null object
dtypes: int64(1), object(4)
memory usage: 25.1+ KB


In [54]:
df_awards_raw.head()

,motivation,num_recipients,recipients,title,year
0,The 2019 Nobel Prize in Physics has not been a...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
1,None,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
2,for groundbreaking inventions in the field of ...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
3,Frances H. Arnold “for the directed evolution ...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
4,The 2018 Nobel Prize in Literature has not bee...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."


In [56]:
# EDA
# df_awards_raw.year.min()

In [57]:
df_awards_raw.num_recipients.value_counts()

0    641
Name: num_recipients, dtype: int64

In [59]:
df_awards_raw[df_awards_raw.num_recipients == 0].head()

,motivation,num_recipients,recipients,title,year
0,The 2019 Nobel Prize in Physics has not been a...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
1,None,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
2,for groundbreaking inventions in the field of ...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
3,Frances H. Arnold “for the directed evolution ...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."
4,The 2018 Nobel Prize in Literature has not bee...,0,[],All Nobel Prizes,"<div class=""by_year""> <h3>2019</h3> <h3><a hre..."


In [ ]:
# df_awards_past = df_awards_raw[df_awards_raw.year != 2018]
# df_awards_past.info()

In [60]:
# df_awards_past[df_awards_past.motivation.isnull()].head()

NameError: name 'df_awards_past' is not defined

In [61]:
# Other structured data formats: JSON and CSV

In [ ]:
# csv
df_awards_past.to_csv('awards.csv', index=False)
with open('awards.csv', 'r') as f:
    print(f.read()[:1000])

In [ ]:
pd.read_csv('awards.csv').recipients.iloc[20]

In [ ]:
# json
df_awards_past.to_json('awards.json', orient='records')

with open('awards.json', 'r') as f:
    print(f.read()[:1000])

In [ ]:
# pickle
df_awards_past.to_pickle('awards.pkl', protocol=0)
with open('awards.pkl', 'r', encoding='latin1') as f:
    print(f.read()[:200])

In [ ]:
# formatted data output
for idx, row in df_awards_past.sort_values('year').iterrows():
    if 'Physics' in row['title']:
        print('{}: {}'.format(
            row['year'],
            ', '.join(row['recipients'])))

In [ ]:
# https://harvard-iacs.github.io/2018-CS109A/sections/section-1/solutions/